In [2]:
!pip install pygris

/bin/bash: pip: command not found


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import pygris
from pygris.data import get_census
from pygris import tracts
from pygris.utils import erase_water

In [4]:
# just NYC tracts

nyc_counties = ["New York", "Bronx", "Richmond", 
                "Kings", "Queens"]

nyc_tracts = tracts(state = "NY", county = nyc_counties, cb = True,
                    year = 2021, cache = True)

Using FIPS code '36' for input 'NY'
Using FIPS code '061' for input 'New York'
Using FIPS code '005' for input 'Bronx'
Using FIPS code '085' for input 'Richmond'
Using FIPS code '047' for input 'Kings'
Using FIPS code '081' for input 'Queens'


In [5]:
# population estimates for under 5 (ACS api call using get_census)

ny_under5 = get_census(dataset = "acs/acs5/profile",
                        variables = "DP05_0005E", # variable name found at https://api.census.gov/data/2021/acs/acs5/profile/variables.html
                        year = 2021,
                        params = {
                          "for": "tract:*",
                          "in": "state:36"},
                        guess_dtypes = True,
                        return_geoid = True)

# percents (ACS api call using get_census)

ny_under5_p = get_census(dataset = "acs/acs5/profile",
                        variables = "DP05_0005PE", # variable name found at https://api.census.gov/data/2021/acs/acs5/profile/variables.html
                        year = 2021,
                        params = {
                          "for": "tract:*",
                          "in": "state:36"},
                        guess_dtypes = True,
                        return_geoid = True)

In [8]:
# building/ cleaning df

ny_under5 = ny_under5.rename(columns={'DP05_0005E':'Estimated Pop. Under 5'}) # renaming column
ny_under5['Percent Under 5'] = ny_under5_p['DP05_0005PE'] # adding percent column to main df

ny_under5 = ny_under5[['Estimated Pop. Under 5','Percent Under 5','GEOID']] # reordering the columns

In [9]:
# df of just NYC census tract data

nyc_under5_merged = nyc_tracts.merge(ny_under5, how = "inner", on = "GEOID") # merging with nyc_tracts to get info just for NYC tracts

nyc_under5_erase = erase_water(nyc_under5_merged, area_threshold = 0.9) # erasing water

In [12]:
nyc_under5_erase = nyc_under5_erase[['COUNTYFP','TRACTCE','GEOID','geometry','Estimated Pop. Under 5','Percent Under 5']]

nyc_under5_erase.to_csv('under5-by-tract_nyc') # downloading as csv